In [3]:
library(bnlearn)
library(tidyverse)
library(graph)
library(gRbase)
library(gRain)

── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.0     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following object is masked from ‘package:b

In [4]:
Data<-read_csv("./data/output/Data_RASFF_EMA_new.csv") 

Data<-Data %>% filter(Data_source=="RASFF")%>% select(-Data_source) %>% rename(Fraud=Product_fraudtype,Product=Product_category,Notified=`Ct_country-name`,Origin=`Or_country-name`)  %>% filter(Year<2014)


Data<-Data %>% mutate_if(is.numeric,as.character)

Data[is.na(Data)]<-"Missing data"



set.seed(0)


Sampled_rows<-sample(1:length(Data$Fraud),ceiling(length(Data$Fraud)*0.2),rep=FALSE)

BNdata_train<-Data[-Sampled_rows,]

BNdata_test<-Data[Sampled_rows,]

BNdata_train<-BNdata_train %>%  mutate_if(is.character,as.factor)

BNdata_test<-BNdata_test %>%  mutate_if(is.character,as.factor)

write.csv(BNdata_train,"./data/output/BNdata_train_2008_2013.csv", row.names = FALSE)
write.csv(BNdata_test,"./data/output/BNdata_test_2008_2013.csv", row.names = FALSE)

Rows: 441 Columns: 6

── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Data_source, Product_category, Or_country-name, Ct_country-name, Pr...
dbl (1): Year


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [2]:
BNdata_train<-as.data.frame(BNdata_train)
BNdata_test<- as.data.frame(BNdata_test)

head(BNdata_train)

#structure leaning
BN_structure<- tree.bayes(BNdata_train,'Fraud')

## test other learning algorithms #

#BN_structure<-gs(BNdata_train)  ### Grow-Shrink (GS) Constraint-based algorithm
#BN_structure<-iamb(BNdata_train)### The incremental association Markov boundary algorithm
#BN_structure<-hc(BNdata_train)### Hill climbing algorithm

graphics::plot(BN_structure)
#graphviz.plot(BN_structure, shape = "ellipse")

ERROR: Error in as.data.frame(BNdata_train): object 'BNdata_train' not found


In [4]:
BIC(BN_structure, BNdata_train)

[1] -12631.96

In [5]:
## parameter learning
BN_trained<-bn.fit(x=BN_structure,data=BNdata_train,method = "bayes")

## convert to gRain
fitted.grain = as.grain(BN_trained)

In [6]:
#Retrieve marginal probability distribution of each node
Mprobs = querygrain(fitted.grain, type = "marginal")
Mprobs 

$Fraud
Fraud
Smuggling-Mislabelling-Origin Masking                 Substitution-Dilution 
                            0.5277778                             0.4722222 

$Product
Product
                 Alcoholic              Confectionary 
               0.006515775                0.006515775 
                     Dairy                       Eggs 
               0.080589849                0.043552812 
              Fish_Seafood Food_additives_supplements 
               0.123799726                0.018861454 
            Grain_products               Herbs_spices 
               0.012688615                0.012688615 
                     Honey                       Meat 
               0.012688615                0.407750343 
               Mixed_foods                       Nuts 
               0.012688615                0.018861454 
                 Oils_Fats                    Poultry 
               0.025034294                0.025034294 
            Prepared_meals                     Snacks 
               0.129972565                0.006515775 
               Soups_sauce                 Vegetables 
               0.018861454                0.037379973 

$Origin
Origin
       Austria        Belarus        Belgium         Brazil       Bulgaria 
    0.00630144     0.00630144     0.01864712     0.01864712     0.00630144 
         China       Colombia        Croatia Czech Republic        Ecuador 
    0.19765947     0.00630144     0.01247428     0.00630144     0.00630144 
         Egypt        Estonia Faeroe Islands        Finland         France 
    0.01247428     0.00630144     0.00630144     0.00630144     0.03716564 
        Gambia        Germany          Ghana         Guinea        Hungary 
    0.01864712     0.04951132     0.00630144     0.00630144     0.01864712 
         India        Ireland          Italy         Jordan         Latvia 
    0.01864712     0.01864712     0.05568416     0.00630144     0.00630144 
       Lebanon  Liechtenstein      Lithuania        Morocco    Netherlands 
    0.01247428     0.00630144     0.00630144     0.02481996     0.01864712 
      Pakistan           Peru    Philippines         Poland       Portugal 
    0.01247428     0.00630144     0.04333848     0.14210391     0.01247428 
       Romania   Saudi Arabia    South Korea          Spain         Sweden 
    0.02481996     0.00630144     0.00630144     0.00630144     0.01864712 
        Taiwan         Turkey        Ukraine United Kingdom  United States 
    0.01247428     0.00630144     0.03099280     0.00630144     0.00630144 
       Uruguay        Vietnam unknown origin 
    0.01247428     0.00630144     0.01247428 

$Year
Year
      2008       2009       2010       2011       2012       2013 
0.11831276 0.06893004 0.13683128 0.12448560 0.07510288 0.47633745 

$Notified
Notified
       Austria        Belgium       Bulgaria         Cyprus Czech Republic 
   0.012583096    0.018755935    0.012583096    0.012583096    0.018755935 
       Denmark        Estonia        Finland         France        Germany 
   0.037274454    0.006410256    0.006410256    0.043447293    0.105175689 
        Greece        Hungary        Ireland          Italy         Latvia 
   0.068138651    0.012583096    0.024928775    0.185422602    0.018755935 
     Lithuania     Luxembourg    Netherlands         Norway         Poland 
   0.006410256    0.012583096    0.061965812    0.037274454    0.055792972 
      Portugal        Romania       Slovenia          Spain         Sweden 
   0.012583096    0.031101614    0.049620133    0.049620133    0.037274454 
United Kingdom 
   0.061965812

In [7]:
## internal validation: confusion matrix
pred_in <- predict(fitted.grain , response = c("Fraud"), newdata = BNdata_train, type = "class")

pred_in <- as.data.frame(pred_in)
table(BNdata_train$Fraud,pred_in$Fraud)
Detection_probability_inter<-sum(diag(table(BNdata_train$Fraud,pred_in$Fraud)))/
  sum(table(BNdata_train$Fraud,pred_in$Fraud))
Detection_probability_inter

                                       
                                        Smuggling-Mislabelling-Origin Masking
  Smuggling-Mislabelling-Origin Masking                                    84
  Substitution-Dilution                                                     0
                                       
                                        Substitution-Dilution
  Smuggling-Mislabelling-Origin Masking                     1
  Substitution-Dilution                                    76

[1] 0.9937888

In [8]:
## external validation: confusion matrix
pred_ex <- predict(fitted.grain, response = c("Fraud"), newdata=BNdata_test, type = "class")

pred_ex <- as.data.frame(pred_ex)

table(BNdata_test$Fraud,pred_ex$Fraud)
Detection_probability_exter<-sum(diag(table(BNdata_test$Fraud,pred_ex$Fraud)))/sum(table(BNdata_test$Fraud,pred_ex$Fraud))
Detection_probability_exter

                                       
                                        Smuggling-Mislabelling-Origin Masking
  Smuggling-Mislabelling-Origin Masking                                    17
  Substitution-Dilution                                                     1
                                       
                                        Substitution-Dilution
  Smuggling-Mislabelling-Origin Masking                     3
  Substitution-Dilution                                    20

[1] 0.902439